In [1]:
import csv
import re
import requests
import os

from collections import defaultdict, namedtuple
from functools import reduce

from bs4 import BeautifulSoup as BS
import numpy as np
import pandas as pd
import simplejson as json
from tenacity import retry, stop_after_attempt
import urllib.request

In [2]:
session = requests.Session()
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com.tw/",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) "\
                  "Chrome/69.0.3497.92 Safari/537.36"
}

In [4]:
@retry(stop=stop_after_attempt(3))
def get_bsObj(url):
    req = session.get(url, headers=headers)
    bsObj = BS(req.text, "html.parser")
    return bsObj

In [5]:
def wos_bab_url_generator(end):
    
    url = "https://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=7&SID=D3unXcbM89Q7qPUjJQE&page=1&doc=%d"
    for i in range(1, end+1):
        yield url % i

In [9]:
# bs = get_bsObj("https://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=16&SID=D4LioZoIMCSfDCJoZGJ&page=1&doc=3")
bs = get_bsObj("https://apps.webofknowledge.com/full_record.do?product=WOS&search_mode=GeneralSearch&qid=7&SID=D3unXcbM89Q7qPUjJQE&page=1&doc=3")

In [10]:
def extract_article_meta(bsObj):
    meta = {}
    meta['title'] = bsObj.find(class_='title').value.text
    meta['authors'] = []
    
    for author_obj in bsObj.find('p', class_='FR_field').find_all('a'):
        name = author_obj.text
        href_statements = author_obj['href'].split('?')[-1]
        for statement in href_statements.split('&'):
            variable, value = statement.split('=')
            if variable == 'daisIds':
                meta['authors'].append({'name': name, 'ID': int(value)})
    
    meta['year'] = bsObj.find('input', {"name": "00N70000002BdnY"})['value']
    meta['citing url'] = "https://apps.webofknowledge.com/" + \
                         bsObj.find("a", class_="view-all-link snowplow-view-all-in-cited-references-page-top")['href']
#     meta['citing meta'] = generate_citing_url(meta['citing url'])
    meta['cited url'] = "https://apps.webofknowledge.com/" + \
                        bsObj.find("a", class_="snowplow-citation-network-times-cited-count-link")['href']
    
    return meta

extract_article_meta(bs)

AttributeError: 'NoneType' object has no attribute 'value'

In [ ]:
def generate_citing_url(url):
    
    bs = get_bsObj(url)
    doc_num = int(bs.find('p', class_='NEWpageTitle').text.split()[-1])
    page_num = (doc_num - 1) // 30 + 1
    
    metas = []
    for i in range(1, page_num+1):
        page_url = url[:-1] + str(i)
        bs = get_bsObj(url)
        entries = bs.find_all('div', class_='search-results-item')
        for entry in entries:
            e = entry.find('a', class_='smallV110 snowplow-full-record')
            if e is None:
                continue
            link = "https://apps.webofknowledge.com/" + e['href']
            print(link)
            bs = get_bsObj(link)
            if bs is None:
                continue
            meta = extract_article_meta(bs)
            print(meta)
    metas.append(meta)
    
    
    return metas

metas = generate_citing_url('https://apps.webofknowledge.com/summary.do?product=WOS&parentProduct=WOS&search_mode=CitedRefList&parentQid=7&parentDoc=3&qid=8&SID=D3unXcbM89Q7qPUjJQE&colName=WOS&page=1')

In [ ]:
<a  href="/CitedFullRecord.do?product=WOS&amp;colName=WOS&amp;SID=D3unXcbM89Q7qPUjJQE&amp;search_mode=CitedFullRecord&amp;isickref=WOS:A1995TH57500003" tabindex="0" oncontextmenu="javascript:return IsAllowedRightClick(this);" hasautosubmit="true"><span class="reference-title">
<value lang_id="">THE 2-STAGE MODEL OF LEXICAL RETRIEVAL - EVIDENCE FROM A CASE OF ANOMIA WITH SELECTIVE PRESERVATION OF GRAMMATICAL GENDER</value>
</span></a>

In [ ]:
article_metas = []

for url in wos_bab_url_generator(14994):
    bsObj = get_bsObj(url)
    meta = extract_article_meta(bsObj)
    article_metas.append(meta)

In [ ]:
article_metas

In [14]:
lambda x: (x=10 if x > 5 else 6; x+6)

SyntaxError: invalid syntax (<ipython-input-14-fc2ddffe54d7>, line 1)

In [15]:
def process_df(df):

    df = df[['PT', 'Z2', 'PD']].reset_index(drop=True).dropna()
    df['PD'] = df['PD'].astype(np.int32)
    df = df.rename(columns={'PT': 'Authors', 'Z2': 'Title', 'PD':'Year'})
    return df

data_dir = 'data'
bab_journals = [os.path.join(data_dir, 'savedrecs.txt'),
                os.path.join(data_dir, 'savedrecs-1.txt')]

df_s = [pd.read_csv(j, sep='\t') for j in bab_journals]
df = reduce(lambda x, y: x.append(y), [process_df(df) for df in df_s])

In [16]:
df

,Authors,Title,Year
0,"Aarts, Esther; El Aidy, Sahar",Increasing reproducibility and interpretabilit...,2019
1,"Abbs, JH; DePaul, R",Motor cortex fields and speech movements: Simp...,1998
2,"ABBS, JH",A SPEECH-MOTOR-SYSTEM PERSPECTIVE ON NERVOUS-S...,1982
3,"Abdallah, Chadi G.; Averill, Lynnette A.; Krys...",Glutamate and norepinephrine interaction: Rele...,2016
4,"Abdi, H; Valentin, D; Edelman, BG",Eigenfeatures as intermediate-level representa...,1998
5,"ABELSON, RP",COMMENTARY POINTS,1983
6,"ABELSON, RP",GOING AFTER PARRY,1981
7,"ABELSON, RP",SEARLE ARGUMENT IS JUST A SET OF CHINESE SYMBOLS,1980
8,"ABELSON, RP",IMAGINING THE PURPOSE OF IMAGERY,1979
9,"ABITBOL, MM",THE MULTIPLE OBSTACLES TO ENCEPHALIZATION,1990


In [29]:
df_cited = pd.read_csv('data/sample-cited.txt', sep='\t')
df_cited

,PT,AU,BA,CA,GP,RI,OI,BE,Z2,TI,...,ZS,Z9,U1,U2,SN,EI,BN,UT,PM,Unnamed: 56
0,J,"AHN, WK; KALISH, CW; MEDIN, DL; GELMAN, SA",NaN,NaN,NaN,NaN,"Kalish, Charles/0000-0002-4463-000X; Gelman, S...",NaN,NaN,THE ROLE OF COVARIATION VERSUS MECHANISM INFOR...,...,NaN,NaN,NaN,NaN,0010-0277,NaN,NaN,WOS:A1995QK95600002,7720361.0,NaN
1,J,"Ahn, WK; Bailenson, J",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Causal attribution as a search for underlying ...,...,NaN,NaN,NaN,NaN,0010-0285,NaN,NaN,WOS:A1996VB50000004,8812022.0,NaN
2,J,"ANDERSON, JR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARGUMENTS CONCERNING REPRESENTATIONS FOR MENTA...,...,NaN,NaN,NaN,NaN,0033-295X,NaN,NaN,WOS:A1978FE49700001,NaN,NaN
3,J,"Barsalou, LW; Huttenlocher, J",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Basing categorization on individuals and events,...,NaN,NaN,NaN,NaN,0010-0285,NaN,NaN,WOS:000075598900001,9729903.0,NaN
4,J,"BARSALOU, LW",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"IDEALS, CENTRAL TENDENCY, AND FREQUENCY OF INS...",...,NaN,NaN,NaN,NaN,0278-7393,NaN,NaN,WOS:A1985ARK5300001,2932520.0,NaN
5,J,"BARSALOU, LW",NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD HOC CATEGORIES,...,NaN,NaN,NaN,NaN,0090-502X,NaN,NaN,WOS:A1983QX27500001,6621337.0,NaN
6,J,"BIEDERMAN, I; GERHARDSTEIN, PC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,RECOGNIZING DEPTH-ROTATED OBJECTS - EVIDENCE A...,...,NaN,NaN,NaN,NaN,0096-1523,1939-1277,NaN,WOS:A1993MK81300002,8294886.0,NaN
7,J,"BIEDERMAN, I",NaN,NaN,NaN,NaN,NaN,NaN,NaN,RECOGNITION-BY-COMPONENTS - A THEORY OF HUMAN ...,...,NaN,NaN,NaN,NaN,0033-295X,NaN,NaN,WOS:A1987G747800001,3575582.0,NaN
8,J,"BLACK, JB; TURNER, TJ; BOWER, GH",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POINT OF VIEW IN NARRATIVE COMPREHENSION, MEMO...",...,NaN,NaN,NaN,NaN,0022-5371,NaN,NaN,WOS:A1979GZ81600005,NaN,NaN
9,J,"Bloom, P; Markson, L",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Intention and analogy in children's naming of ...,...,NaN,NaN,NaN,NaN,0956-7976,NaN,NaN,WOS:000078528800007,NaN,NaN


In [35]:
def process_df_cited(df):

    df = df[['AU', 'TI', 'PY']].reset_index(drop=True).dropna()
    df['PY'] = df['PY'].astype(np.int32)
    df = df.rename(columns={'AU': 'Authors', 'TI': 'Title', 'PY':'Year'})
    return df

process_df_cited(df_cited).to_csv('articles.csv')